#Load and store the data

We loaded our data with DataBricks inside a table called: web_Google

In [0]:
import pandas as pd
import time 
import matplotlib.pyplot as plt
import numpy as np 
import pyspark
from pyspark.sql.functions import expr, col, min as spark_min, collect_list, when, size, explode, lit, array_sort, split

In [0]:
#get the raw data and clean it 
df = spark.read.table("web_Google").select('_c0')#
df = df.where(~df['_c0'].contains('#'))
#df.show(10)

# Split the '_c0' column into two columns based on the tab character '\t'
df = df.withColumn("key", split(df["_c0"], "\t").getItem(0))
df = df.withColumn("values", split(df["_c0"], "\t").getItem(1)).drop('_c0')

df.show(10)


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1181126391222697>:2
      1 #get the raw data and clean it 
----> 2 df = spark.read.table("web_Google").select('_c0')#
      3 df = df.where(~df['_c0'].contains('#'))
      4 #df.show(10)
      5 
      6 # Split the '_c0' column into two columns based on the tab character '\t'

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:473, in DataFrameReader.table(self, tableName)
    439 def table(self, tableName: str) -> "DataFrame":
    440     """Returns

In [0]:
# Select the "key" and "value" columns from the DataFrame
rdd = df.select("key", "values").rdd.map(lambda x: [int(x[0]), int(x[1])])
rdd.take(10)

#Connected Component Computation 



##RDD implementation


In [0]:
#cf iterate 
def ccf_iterate_rdd(rdd):
    rdd = rdd.union(rdd.map(lambda x: (x[1], x[0])))
    rdd = rdd.groupByKey()

    def new_pairs(key, values):
        min_val = key
        value_list = []
        new_rdd = []
        c = 0
        for value in values:
            if value < min_val:
                min_val = value
            value_list.append(value)

        if min_val < key:
            new_rdd = [(key, min_val)]
            for value in value_list:
                if not min_val == value:
                    c =+1
                    new_rdd.append((value, min_val))
        
        return (key, new_rdd, c)

    rdd = rdd.map(lambda x: new_pairs(x[0], x[1]))
    c = rdd.map(lambda x: x[2]).sum()

    rdd = rdd.flatMap(lambda x: x[1])

    return rdd, c

def ccf_dedup_rdd(rdd):
    #remove duplicate 
    rdd = rdd.map(lambda x: (x, None))
    rdd = rdd.groupByKey()
    rdd = rdd.map(lambda x: x[0])
    return rdd

#ccf iterate sorting
def ccf_iterate_sorting_rdd(rdd):
    rdd = rdd.union(rdd.map(lambda x: (x[1], x[0])))
    rdd = rdd.groupByKey().mapValues(lambda x: sorted(x))

    def new_pairs(key, values):
        min_val = values[0]
        new_rdd = []
        c = 0
        if min_val < key:
            new_rdd.append((key, min_val))
            for value in values:
                if not value == min_val:
                    c =+1
                    new_rdd.append((value, min_val))

        return (key, new_rdd, c)

    rdd = rdd.map(lambda x: new_pairs(x[0], x[1]))
    c = rdd.map(lambda x: x[2]).sum()

    rdd = rdd.flatMap(lambda x: x[1])

    return rdd, c



##DataFrame implementation

In [0]:
def ccf_iterate_df(df):

    df = df.union(df.select(["values", "key"]))
    df = df.groupBy("key").agg(spark_min("values").alias("min_val"), collect_list("values").alias("values"))
    df = df.filter(col("min_val") < col("key"))

    #emit 1
    df_min = df.select(col("key"), col("min_val").alias("values"))

    df = df.select("key", "min_val", explode("values").alias("values")).filter(col("min_val") != col("values"))

    #emit 2
    df_new_pairs = df.select(col("values").alias("key"), col("min_val").alias("values"))

    df_result = df_min.union(df_new_pairs)
    c = df_new_pairs.count()

    return df_result, c


def ccf_dedup_df(df):
    df = df.dropDuplicates()
    return df

def ccf_iterate_sorting_df(df):

    df = df.union(df.select(["values", "key"]))
    df = df.groupBy("key").agg(array_sort(collect_list("values")).alias("value_list"))
    df = df.withColumn("min_val", col("value_list")[0])
    
    df = df.filter(col("min_val") < col("key"))

    #emit1
    df_min = df.select(col("key"), col("min_val").alias("values"))
    
    #emit 2
    df_new_pairs = df.withColumn("values", explode(col("value_list")))
    df_new_pairs = df_new_pairs.filter(col("min_val") < col("values")).select(col("values").alias("key"), col("min_val").alias("values"))
    
    c = df_new_pairs.count()
    # Union the two DataFrames
    df_result = df_min.union(df_new_pairs)

    return df_result, c



##Main Loop

In [0]:
def main_loop_rdd(rdd, max_iteration, sorting=False):
    c = 1
    k = 0

    start_time = time.time()
    while c > 0:
        if not sorting:
            rdd, c = ccf_iterate_rdd(rdd)
        else: 
            rdd, c = ccf_iterate_sorting_rdd(rdd)
        rdd = ccf_dedup_rdd(rdd)
        #print("iter", k)
        k = k + 1
        #print('Iteration', k)
        if k > max_iteration:

            print("EXCEEDED MAX NUMBER OF ITERATION")
            break 
    #print("ended while")
    time_ccf = time.time() - start_time

    return rdd, time_ccf, k

def main_loop_df(df, max_iteration, sorting=False):
    c = 1
    k = 0

    start_time = time.time()
    while c > 0:
        if not sorting:
            df, c = ccf_iterate_df(df)
        else: 
            df, c = ccf_iterate_sorting_df(df)
        df = ccf_dedup_df(df)
        #print("iter", k)
        k = k + 1
        #print('Iteration df', k)
        if k > max_iteration:
            print("EXCEEDED MAX NUMBER OF ITERATION")
            break 
    #print("Ended while")
    time_ccf = time.time() - start_time

    return df, time_ccf, k


##Sanity Check

We run a test on the graph presented in the white paper. It should take 4 iterations to parse it. 
And the out put should be: (2,1), (3,1), (4,1), (5,1), (7,6), (8,6)

In [0]:
#sanity check
rdd_test = sc.parallelize([(1, 2), (2, 3), (2, 4), (4, 5), (6, 7), (7, 8)])
df_test = rdd_test.toDF(["key", "values"])

df_out, time_, k = main_loop_df(df_test, 10, True)

print("Iterations: ", k)
#df_out.show()

In [0]:
rdd_out, time_, k = main_loop_rdd(rdd_test, 10, False)

print("Iterations: ", k)
rdd_out.take(10)

#Experimental Analysis
Experimental analysis comparing the RDD and DataFrame versions
is conducted on graphs of increasing size


##Run modulations of our program on graphs of increasing size 

In [0]:
results = {'DataType': [], 'Sorting': [], 'Size': [],'Iterations': [], 'Time(s)': []}

max_iterate = 10000
data_size = 1000


#increase the size of the rrd to measure impact in time 
for n in np.arange(1, data_size, step=100):

    df_sample = df.limit(int(n))
    rdd_sample = df_sample.rdd

    #rdd without sorting
    #print("start iter rdd no sort")
    rdd_out, time_ccf, k = main_loop_rdd(rdd_sample, max_iterate, False)

    results['DataType'].append('RDD')
    results['Sorting'].append('False')
    results['Iterations'].append(k)
    results['Time(s)'].append(time_ccf)
    results["Size"].append(n)

    #rdd with sorting
    #print("start iter rdd sort")
    rdd_out, time_ccf, k = main_loop_rdd(rdd_sample, max_iterate, True)

    results['DataType'].append('RDD')
    results['Sorting'].append('True')
    results['Iterations'].append(k)
    results['Time(s)'].append(time_ccf)
    results["Size"].append(n)

    #DF without sorting
    #print("start iter df")
    df_out, time_ccf, k = main_loop_df(df_sample, max_iterate, False)

    results['DataType'].append('DataFrame')
    results['Sorting'].append('False')
    results['Iterations'].append(k)
    results['Time(s)'].append(time_ccf)
    results["Size"].append(n)

    #DF with sorting
    #print("start iter df sort")
    df_out, time_ccf, k = main_loop_df(df_sample, max_iterate, True)

    results['DataType'].append('DataFrame')
    results['Sorting'].append('True')
    results['Iterations'].append(k)
    results['Time(s)'].append(time_ccf)
    results["Size"].append(n)

    print("Sample size", n, "parsed")


##Display results


In [0]:
results_df = pd.DataFrame(results)
results_df

In [0]:
#visual analysis 

# Define a dictionary to map unique values of 'DataType' to colors
data_type_colors = {'RDD': 'blue', 'DataFrame': 'green'}

# Define a dictionary to map unique values of 'Sort' to marker shapes
sort_markers = {'True': 'o', 'False': 'x'}

# Create a figure and axis for the plot
fig, ax = plt.subplots()

# Iterate through unique combinations of 'DataType' and 'Sort'
for data_type, group1 in results_df.groupby('DataType'):
    for sort, group2 in group1.groupby('Sorting'):
        marker = sort_markers.get(sort, 'o')
        color = data_type_colors.get(data_type, 'gray')
        label = f'{data_type}, {sort}'
        ax.scatter(group2['Size'], group2['Time(s)'], c=color, marker=marker, label=label)

# Set labels and legend
ax.set_xlabel('Size')
ax.set_ylabel('Time')
ax.legend(loc='upper left')

# Show the plot
plt.show()